### This notebook is concerned to ML Bootcamp Competition 2023 for Ukrainians which was carried out in August 2023
### Instruments used in this NN are purely got from free bootcamp "Introduction to TensorFlow for Artificial Intelligence, Machine Learning, and Deep Learning"
### Workout plan:
##### 1.Explore images and get aquainted with yoga poses
##### 2.Preprocess images: remove bachground and crop human pose to the centre of image
##### 3.Workout with simple NN moder in order to get approximate hyperparameters that gives satisfactory results
##### 4.Implement Inception model and train it with several variations of hyperparameters (got in i.3) in order to get different prediction results
##### 5.Concatenate those predictions (got in i.4) using soft votes and predict test images
##### 6.Save model to file

In [20]:

import os
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image, ImageChops
from rembg import remove
from sklearn.model_selection import train_test_split

## define direcotry with images and labels
TRAIN_IMAGES_DIRECTORY = 'images/train_images/'
TEST_IMAGES_DIRECTORY = 'images/test_images/'
TRAIN_LABELS_CSV = 'train.csv'
RESOLUTION = 200

## function that trim free space at image and position the image in the centre of picture
def trim(im):
    bg = Image.new(im.mode, im.size, im.getpixel((0,0)))
    diff = ImageChops.difference(im, bg)
    diff = ImageChops.add(diff, diff, 2.0, -100)
    bbox = list(diff.getbbox())
    if bbox:
        if bbox[2]-bbox[0] != bbox[3]-bbox[1]:
            add_space = (bbox[2]-bbox[0])-(bbox[3]-bbox[1])
            if add_space > 0:
                bbox[3] += int(add_space/2)
                bbox[1] -= int(add_space/2)
            else:
                bbox[0] += int(add_space/2)
                bbox[2] -= int(add_space/2) 
        return im.crop(bbox)

## function that removes background from pictures and converts them to np.array (4dim format). Returns tuple of numpy array of images and their ids
def img_to_arr_crop(directory, resolution, img_list = [], img_format = '.jpg'):
    res_list = []
    if img_list == []:
        img_list = [i for i in os.listdir(directory) if i[-4:] == img_format]
    for i in img_list:
        curr_image = np.asarray(trim(remove(Image.open(directory+i))).resize((resolution, resolution)))
        if len(curr_image.shape) < 3:
            curr_image = np.asarray(trim(remove(Image.open(directory+i))).resize((resolution, resolution)).convert('RGB'))
        res_list.append(curr_image)
    res_images = np.array(res_list).astype(float)
    return res_images, img_list

## function for plotting fitting results
def plot_learning_results(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(acc))

    plt.plot(epochs, acc, 'r', label='Training accuracy')
    plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.figure()

    plt.plot(epochs, loss, 'r', label='Training Loss')
    plt.plot(epochs, val_loss, 'b', label='Validation Loss')
    plt.title('Training and validation loss')
    plt.legend()
    plt.show()

## function for training a model
def fitting_model(hyperparameters):
    ## define a model and its structure before fitting
    pre_trained_model = tf.keras.applications.InceptionV3(input_shape = (hyperparameters['RESOLUTION'], hyperparameters['RESOLUTION'], 3), 
                                                        include_top = False, 
                                                        weights = 'imagenet')

    for layer in pre_trained_model.layers:
        layer.trainable = False

    last_layer = pre_trained_model.get_layer(hyperparameters['layer'])
    last_output = last_layer.output

    x = tf.keras.layers.Flatten()(last_output)
    x = tf.keras.layers.Dropout(hyperparameters['dropout'])(x)    
    x = tf.keras.layers.Dense(hyperparameters['Dense'], activation='relu')(x)
    x = tf.keras.layers.Dropout(hyperparameters['dropout'])(x)    
    x = tf.keras.layers.Dense (6,activation='softmax')(x)           

    model = tf.keras.Model(pre_trained_model.input, x) 
    loss_f = tf.keras.losses.SparseCategoricalCrossentropy()
    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=hyperparameters['lr']), 
                    loss = loss_f, 
                    metrics = ['accuracy'])

    ## split values on training ang validation

    X_train, X_val, y_train, y_val = train_test_split(training_images[:,:,:,:3], lbl_list, test_size=hyperparameters['val_split'], stratify = lbl_list)

    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1.0/255.0,
                                                                    width_shift_range=hyperparameters['width_shift_range'],
                                                                    height_shift_range=hyperparameters['height_shift_range'],
                                                                    rotation_range=hyperparameters['rotation_range'],
                                                                    shear_range=hyperparameters['shear_range'],
                                                                    zoom_range=hyperparameters['zoom_range'],
                                                                    fill_mode=hyperparameters['fill_mode'],
                                                                    horizontal_flip=hyperparameters['horizontal_flip'],
                                                                    )

    train_generator = train_datagen.flow(x = X_train,
                                        y=y_train,
                                        batch_size=hyperparameters['batch_size_t'],
                                        shuffle = hyperparameters['shuffle'])

    validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1.0/255.0,
                                                                        validation_split=hyperparameters['val_split']
                                                                        )

    validation_generator = validation_datagen.flow(x = X_val,
                                                    y=y_val,
                                                    batch_size=hyperparameters['batch_size_v'],
                                                    shuffle = hyperparameters['shuffle'])

    ## fitting model
    history = model.fit(train_generator,
                        epochs=hyperparameters['epochs'],
                        validation_data=validation_generator)
    return history

# Preprocessing images

In [ ]:
## Create lists of labels and images
df  = pd.read_csv(TRAIN_LABELS_CSV)
img_list = list(df['image_id'])
lbl_list = list(df['class_6'])

## converting images: takes around 2 sec per image
training_images = img_to_arr_crop(TRAIN_IMAGES_DIRECTORY, RESOLUTION, img_list)[0] ## no need to get ids, because it was done on previous step
testing_images, testing_labels = img_to_arr_crop(TEST_IMAGES_DIRECTORY, RESOLUTION)

## show random image before and after processing
test_example_image = np.random.randint(len(training_images))
example_image = img_list[test_example_image]
print('image: ', example_image)
old_img = np.asarray(Image.open(TRAIN_IMAGES_DIRECTORY+example_image))
plt.imshow(old_img.astype('uint8'))
plt.show()
new_img = training_images[test_example_image]
plt.imshow(new_img.astype('uint8'))
plt.show()

# Save & load preprocessing results
#### In case we want to postpone workout

In [18]:
np.save("all_training_images.npy", training_images)
np.save("all_training_labels.npy", np.array(lbl_list))
np.save("all_training_ids.npy", np.array(img_list))
np.save("all_testing_images.npy", testing_images)
np.save("all_testing_labels.npy", np.array(testing_labels))

In [19]:
training_images = np.load("all_training_images.npy")
lbl_list = list(np.load("all_training_labels.npy"))
img_list = list(np.load("all_training_ids.npy"))
testing_images = np.load("all_testing_images.npy")
testing_labels = list(np.load("all_testing_labels.npy"))


## Simple NN workout 
#### with the aim to get satisfying hyperparameters

In [ ]:
hyperparameters = {'val_split':0.15, 
                'RESOLUTION':RESOLUTION, 
                'width_shift_range':0.05, 
                'height_shift_range':0.05,
                'zoom_range':0.05, 
                'fill_mode':'nearest', 
                'horizontal_flip':True,
                'shuffle':True,
                'batch_size_t':32,
                'batch_size_v':8,
                'Conv2D_1':16,
                'Conv2D_2':16,
                'Conv2D_3':16,
                'Dense':128,
                'epochs':50,
                'loss':'sparse_categorical_crossentropy',
                'optimizer':'Adam',
                'lr':0.0005,
                'dropout':0.45}

def train_simple_model(hyperparameters):
    X_train, X_val, y_train, y_val = train_test_split(training_images, lbl_list, test_size=hyperparameters['val_split'], stratify = lbl_list)

    train_datagen = ImageDataGenerator(rescale = 1.0/255.0,
                                         width_shift_range=hyperparameters['width_shift_range'],
                                         height_shift_range=hyperparameters['height_shift_range'],
                                         zoom_range=hyperparameters['zoom_range'],
                                         fill_mode=hyperparameters['fill_mode'],
                                         horizontal_flip=hyperparameters['horizontal_flip'],
                                         )
    train_generator = train_datagen.flow(x=X_train,
                                        y=y_train,
                                        shuffle = hyperparameters['shuffle'],
                                        batch_size=hyperparameters['batch_size_t'])
    validation_datagen = ImageDataGenerator(rescale = 1.0/255.0,
                                              )
    validation_generator = validation_datagen.flow(x=X_val,
                                                  y=y_val,
                                                    shuffle = hyperparameters['shuffle'],
                                                  batch_size=hyperparameters['batch_size_v'])

    model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(hyperparameters['Conv2D_1'],(3,3),activation='relu',input_shape=(hyperparameters['RESOLUTION'],hyperparameters['RESOLUTION'],4)),
      tf.keras.layers.MaxPooling2D(3,3),
      tf.keras.layers.Conv2D(hyperparameters['Conv2D_2'],(3,3),activation='relu'),
      tf.keras.layers.MaxPooling2D(3,3),      
      tf.keras.layers.Conv2D(hyperparameters['Conv2D_3'],(3,3),activation='relu'),
      tf.keras.layers.MaxPooling2D(3,3),
      tf.keras.layers.Dropout(hyperparameters['dropout']),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(hyperparameters['Dense'],activation='relu'),
      tf.keras.layers.Dense(6,activation='softmax')
    ])
    optimizer = tf.keras.optimizers.Adam(learning_rate=hyperparameters['lr'])
    model.summary()
    model.compile(optimizer = optimizer,
                loss = hyperparameters['loss'],
                metrics=['accuracy'])

    history = model.fit(train_generator,
                        epochs=hyperparameters['epochs'],
                        validation_data=validation_generator)
    
    print(hyperparameters)

    plot_learning_results(history)
    
    return model

train_simple_model(hyperparameters).save('model_pre_0.keras')

## Finetuning and voting for best result 
#### Get 3 different results and concatenating them with soft voting 

In [ ]:
## setting approximate hyperparameters were got after some stages of training 
hyperparameters={'val_split':0.1, 
                'layer':'mixed7',
                'RESOLUTION':RESOLUTION, 
                'width_shift_range':0.1, 
                'height_shift_range':0.1,
                'zoom_range':0.1,
                'shear_range':0.1,
                'rotation_range':10,
                'fill_mode':'nearest', 
                'horizontal_flip':True,
                'shuffle':True,
                'batch_size_t':8,
                'batch_size_v':4,
                'Conv2D_1':16,
                'Dense':256,
                'epochs':20,
                'loss':'CategoricalCrossentropy',
                'optimizer':'Adam',
                'lr':0.0001,
                'dropout':0.5}

history = fitting_model(hyperparameters)

## show fitting results and get predictions
print('iter_1: ',hyperparameters)

plot_learning_results(history)

predictions_iter_1 = model.predict(testing_images[:,:,:,:3])

model.save('model_iter_1.keras')

In [ ]:
## setting hyperparameters were got after some stages of training 
hyperparameters={'val_split':0.1, 
                'layer':'mixed7',
                'RESOLUTION':RESOLUTION, 
                'width_shift_range':0.07, 
                'height_shift_range':0.07,
                'zoom_range':0.07,
                'shear_range':0.07,
                'rotation_range':7,
                'fill_mode':'nearest', 
                'horizontal_flip':True,
                'shuffle':True,
                'batch_size_t':16,
                'batch_size_v':4,
                'Conv2D_1':32,
                'Dense':128,
                'epochs':15,
                'loss':'CategoricalCrossentropy',
                'optimizer':'Adam',
                'lr':0.0003,
                'dropout':0.3}

history = fitting_model(hyperparameters)

print('iter_2: ',hyperparameters)

plot_learning_results(history)

predictions_iter_2 = model.predict(testing_images[:,:,:,:3])

model.save('model_iter_2.keras')

In [ ]:
## setting hyperparameters were got after some stages of training 
hyperparameters={'val_split':0.1, 
                'layer':'mixed7',
                'RESOLUTION':RESOLUTION, 
                'width_shift_range':0.15, 
                'height_shift_range':0.15,
                'zoom_range':0.15,
                'shear_range':0.15,
                'rotation_range':15,
                'fill_mode':'nearest', 
                'horizontal_flip':True,
                'shuffle':True,
                'batch_size_t':32,
                'batch_size_v':4,
                'Conv2D_1':32,
                'Dense':512,
                'epochs':15,
                'loss':'CategoricalCrossentropy',
                'optimizer':'Adam',
                'lr':0.0001,
                'dropout':0.5}

history = fitting_model(hyperparameters)

print('iter_3: ',hyperparameters)

plot_learning_results(history)

predictions_iter_3 = model.predict(testing_images[:,:,:,:3])

model.save('model_iter_3.keras')

## Making predictions

In [ ]:
predictions = predictions_iter_1 + predictions_iter_2 + predictions_iter_3
df = pd.DataFrame(list(np.argmax(predictions, axis=1)), index=testing_labels).reset_index()
df.columns = ['image_id','class_6']
df.to_csv('submission.csv', index=False)